In [10]:
# not final version

import pandas as pd
import numpy as np
import datetime
import os
import gc
os.getcwd()

'/home/jian/Projects/Garanimal'

In [11]:
df_input=pd.ExcelFile("./Garanimals_Sales_Analysis_JC_1105.xlsx")
df_input.sheet_names

['Sales by Zip Code',
 'histagram kid by store',
 '10 mile zip',
 'histogram kid by TA',
 'Store_View',
 'TA_View',
 'controltest',
 'Sheet6']

In [12]:
df_input=df_input.parse("Sheet6")
df_input.head(2)

,TA,Zip only in 1 TA,Total zip,#zip overlap in other TA,overlap rate,store_count,zip_count,Kid_Sales_Tier,Zip avg- % to Total Sales *100,kids_0_9_census
0,6,161,161,0,0.0,22,161,H1,0.805756,404340.0
1,185,9,9,0,0.0,3,9,M4,0.012066,3200.0


In [13]:
df_input=df_input[df_input['overlap rate']<=0.05]
df_input=df_input[df_input['Kid_Sales_Tier'].isin(['H1','M2','M3'])]
df_input=df_input.rename(columns={"Zip avg- % to Total Sales *100":"sales_field"})
df_input=df_input[['TA','store_count','zip_count','Kid_Sales_Tier','sales_field','kids_0_9_census']]
df_input.shape

(614, 6)

In [14]:
df_input_H1=df_input[df_input['Kid_Sales_Tier']=="H1"]
df_input_M23=df_input[(df_input['Kid_Sales_Tier']=="M2") | (df_input['Kid_Sales_Tier']=="M3")]

In [15]:
dict_1_store_count=df_input.set_index("TA").to_dict()["store_count"]
dict_2_zip_count=df_input.set_index("TA").to_dict()["zip_count"]
dict_3_sales_field=df_input.set_index("TA").to_dict()["sales_field"]
dict_4_kid=df_input.set_index("TA").to_dict()["kids_0_9_census"]

list_ta_H1=df_input_H1['TA'].tolist()
list_ta_M23=df_input_M23['TA'].tolist()


In [16]:
# Rules
'''
1. The some count of TAs combined: 1 to 1; 2 to 2; 3 to 3
2. Absolute % difference sum
'''

'\n1. The some count of TAs combined: 1 to 1; 2 to 2; 3 to 3\n2. Absolute % difference sum\n'

# H1

In [23]:
# 1. no combination
df_input_H1_copy_1=df_input_H1.copy()
df_input_H1_copy_2=df_input_H1.copy()
cols=df_input_H1.columns.tolist()
cols_new=[]
for col in cols:
    cols_new.append(col+"_A")
    cols_new.append(col+"_B")

for col in df_input_H1_copy_1.columns.tolist():
    df_input_H1_copy_1=df_input_H1_copy_1.rename(columns={col:col+"_A"})
    df_input_H1_copy_2=df_input_H1_copy_2.rename(columns={col:col+"_B"})
    
df_input_H1_copy_1['key']=1
df_input_H1_copy_2['key']=1

df_input_H1_1_set=pd.merge(df_input_H1_copy_1,df_input_H1_copy_2,on="key",how="outer")
df_input_H1_1_set=df_input_H1_1_set[cols_new]
df_input_H1_1_set=df_input_H1_1_set[df_input_H1_1_set['TA_A']!=df_input_H1_1_set['TA_B']]
df_input_H1_1_set=df_input_H1_1_set[(df_input_H1_1_set['store_count_A']>=9) & (df_input_H1_1_set['store_count_B']>=9)]
del df_input_H1_1_set['Kid_Sales_Tier_A']
del df_input_H1_1_set['Kid_Sales_Tier_B']

cols_metrics=[x for x in cols if x not in ['TA','Kid_Sales_Tier']]
df_input_H1_1_set.head(2)
# df_input_H1_1_set['score_1']=0
for col_calulate in cols_metrics:
    df_input_H1_1_set['diff_'+col_calulate]=df_input_H1_1_set[col_calulate+"_A"]/df_input_H1_1_set[col_calulate+"_B"]-1
    df_input_H1_1_set['diff_'+col_calulate]=df_input_H1_1_set['diff_'+col_calulate].apply(abs)
'''
    if "count" in col_calulate:
        df_input_H1_1_set['score_1']=df_input_H1_1_set['score_1']+df_input_H1_1_set['diff_'+col_calulate]*0.1667/2
    else:
        df_input_H1_1_set['score_1']=df_input_H1_1_set['score_1']+df_input_H1_1_set['diff_'+col_calulate]*0.3333
'''

df_input_H1_1_set['score_1']=df_input_H1_1_set['diff_sales_field']*0.5+df_input_H1_1_set['diff_kids_0_9_census']*0.35+df_input_H1_1_set['diff_zip_count']*0.15

df_input_H1_1_set=df_input_H1_1_set.sort_values("score_1")

In [24]:
df_test=df_input_H1_1_set[df_input_H1_1_set['store_count_A']==df_input_H1_1_set['store_count_B']]
df_test=df_test.reset_index()
del df_test['index']
df_test.head(20)

,TA_A,TA_B,store_count_A,store_count_B,zip_count_A,zip_count_B,sales_field_A,sales_field_B,kids_0_9_census_A,kids_0_9_census_B,diff_store_count,diff_zip_count,diff_sales_field,diff_kids_0_9_census,score_1
0,63,61,9,9,35,48,0.239395,0.238364,65881.687991,62755.286620,0.0,0.270833,0.004325,0.049819,0.060224
1,61,63,9,9,48,35,0.238364,0.239395,62755.286620,65881.687991,0.0,0.371429,0.004307,0.047455,0.074477
2,56,53,10,10,54,62,0.208467,0.220456,82529.000000,92097.000000,0.0,0.129032,0.054383,0.103890,0.082908
3,53,56,10,10,62,54,0.220456,0.208467,92097.000000,82529.000000,0.0,0.148148,0.057510,0.115935,0.091555
4,44,48,11,11,62,54,0.267459,0.271779,68059.000000,84208.000000,0.0,0.148148,0.015896,0.191775,0.097292
5,45,44,11,11,47,62,0.236397,0.267459,67472.000000,68059.000000,0.0,0.241935,0.116139,0.008625,0.097379
6,52,54,10,10,79,61,0.467516,0.463208,155928.370991,186751.000000,0.0,0.295082,0.009301,0.165047,0.106679
7,54,52,10,10,61,79,0.463208,0.467516,186751.000000,155928.370991,0.0,0.227848,0.009215,0.197672,0.107970
8,48,44,11,11,54,62,0.271779,0.267459,84208.000000,68059.000000,0.0,0.129032,0.016153,0.237279,0.110479
9,44,45,11,11,62,47,0.267459,0.236397,68059.000000,67472.000000,0.0,0.319149,0.131400,0.008700,0.116617


In [10]:
'''
57 43
61 45
39 35
# 63 61
'''

'\n57 43\n61 45\n39 35\n# 63 61\n'

# M23

In [11]:
# Try sum 3 or 4 TAs up to good match
# Total store range 10-13
# Downgraded to 8-9

df_input_M23=df_input_M23.sort_values("store_count",ascending=False)
M23_TA_list=df_input_M23['TA'].tolist()

total_ta_len=len(M23_TA_list)
print(total_ta_len)

print("df_input_M23['store_count'].max()",df_input_M23['store_count'].max())

485
df_input_M23['store_count'].max() 4


In [12]:
dict_store_by_ta_M23=df_input_M23.set_index("TA").to_dict()['store_count']
len(dict_store_by_ta_M23)

485

In [13]:
df_input_M23.shape

(485, 6)

In [14]:
df_input_M23['Kid_Sales_Tier'].unique()

array(['M2', 'M3'], dtype=object)

In [15]:
df_input_M23.groupby("store_count")['TA'].count().to_frame().reset_index().sort_values("TA",ascending=False)

,store_count,TA
0,1,322
1,2,121
2,3,36
3,4,6


# Sum 3 TAs

In [31]:
# For aggregating 4 tas, try sum the array (repeatable) [1,2,3,4] to 10-13 first and then split the df into 4 sets and combine
df_1=pd.DataFrame({"value_1":[1,2,3,4],"key":[0,0,0,0]},index=[x for x in range(4)])
df_2=pd.DataFrame({"value_2":[1,2,3,4],"key":[0,0,0,0]},index=[x for x in range(4)])
df_3=pd.DataFrame({"value_3":[1,2,3,4],"key":[0,0,0,0]},index=[x for x in range(4)])

df_comb_3=pd.merge(df_1,df_2,on="key",how="outer")
df_comb_3=pd.merge(df_comb_3,df_3,on="key",how="outer")

del df_1
del df_2
del df_3
del df_comb_3['key']

print(df_comb_3.shape)
df_comb_3['sum']=df_comb_3['value_1']+df_comb_3['value_2']+df_comb_3['value_3']
df_comb_3=df_comb_3[(df_comb_3['sum']>=8) & (df_comb_3['sum']<=12)]

(64, 3)


In [34]:
df_comb_3['set_count']=df_comb_3[['value_1','value_2','value_3']].values.tolist()
df_comb_3['set_count']=df_comb_3['set_count'].apply(lambda x: sorted(x)).astype(str)
comb_count=df_comb_3['set_count'].unique().tolist()
comb_count=[eval(x) for x in comb_count]

In [43]:
list_ta_1_store=df_input_M23[df_input_M23['store_count']==1]['TA'].tolist()
print(len(list_ta_1_store))

list_ta_2_store=df_input_M23[df_input_M23['store_count']==2]['TA'].tolist()
print(len(list_ta_2_store))

list_ta_3_store=df_input_M23[df_input_M23['store_count']==3]['TA'].tolist()
print(len(list_ta_3_store))

list_ta_4_store=df_input_M23[df_input_M23['store_count']==4]['TA'].tolist()
print(len(list_ta_4_store))


322
121
36
6


In [44]:
globals()["list_ta_"+str(4)+"_store"]

[129, 137, 141, 116, 140, 135]

In [47]:
comb_count

[[1, 3, 4],
 [1, 4, 4],
 [2, 2, 4],
 [2, 3, 3],
 [2, 3, 4],
 [2, 4, 4],
 [3, 3, 3],
 [3, 3, 4],
 [3, 4, 4],
 [4, 4, 4]]

In [46]:
all_ta_pairs_3=[]
for comb in comb_count:
    comb_list=[[x,y,z] for x in globals()["list_ta_"+str(comb[0])+"_store"] for y in globals()["list_ta_"+str(comb[1])+"_store"] for z in globals()["list_ta_"+str(comb[2])+"_store"]]
    all_ta_pairs_3.extend(comb_list)
len(all_ta_pairs_3)     

412242

In [68]:
list_3_df=[]
i_counter=0
for l in all_ta_pairs_3:
    df=df_input_M23[df_input_M23['TA'].isin(l)]
    df['str_ta_list']=str(l)
    df=df.groupby("str_ta_list")['store_count','zip_count','sales_field','kids_0_9_census'].sum().reset_index()
    list_3_df.append(df)
    i_counter+=1
    if i_counter%10000==1:
        print(i_counter,datetime.datetime.now())
df_3_combination_M23=pd.concat(list_3_df)
# del list_3_df
gc.collect()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


1 2019-11-07 13:37:22.596740
10001 2019-11-07 13:38:35.917631
20001 2019-11-07 13:39:48.701367
30001 2019-11-07 13:41:01.568268
40001 2019-11-07 13:42:13.848913
50001 2019-11-07 13:43:27.122655
60001 2019-11-07 13:44:38.795697
70001 2019-11-07 13:45:51.618162
80001 2019-11-07 13:47:03.014755
90001 2019-11-07 13:48:14.530972
100001 2019-11-07 13:49:27.839221
110001 2019-11-07 13:50:40.892742
120001 2019-11-07 13:51:54.524998
130001 2019-11-07 13:53:07.719680
140001 2019-11-07 13:54:21.460203
150001 2019-11-07 13:55:34.255910
160001 2019-11-07 13:56:47.214540
170001 2019-11-07 13:58:00.741139
180001 2019-11-07 13:59:13.478139
190001 2019-11-07 14:00:28.349555
200001 2019-11-07 14:01:41.254573
210001 2019-11-07 14:02:53.931757
220001 2019-11-07 14:04:06.736681
230001 2019-11-07 14:05:19.458033
240001 2019-11-07 14:06:32.842826
250001 2019-11-07 14:07:46.777564
260001 2019-11-07 14:08:59.490196
270001 2019-11-07 14:10:11.851669
280001 2019-11-07 14:11:24.975560
290001 2019-11-07 14:12:37.3

0

In [69]:
df_3_combination_M23.shape

(412242, 5)

In [70]:
df_3_combination_M23.head(2)['str_ta_list'].tolist()[0]

'[887, 194, 129]'

In [71]:
df_3_combination_M23['str_ta_list']=df_3_combination_M23['str_ta_list'].apply(lambda x: sorted(eval(x)))
df_3_combination_M23['str_ta_list']=df_3_combination_M23['str_ta_list'].astype(str)

In [73]:
df_3_combination_M23=df_3_combination_M23.reset_index()
del df_3_combination_M23['index']
df_3_combination_M23=df_3_combination_M23.reset_index()
del df_3_combination_M23['index']



,index,str_ta_list,store_count,zip_count,sales_field,kids_0_9_census
0,0,"[129, 194, 887]",8,40,0.115675,24030.691289
1,1,"[137, 194, 887]",8,31,0.089513,15671.691289


In [75]:
df_3_combination_M23.head(2)

,str_ta_list,store_count,zip_count,sales_field,kids_0_9_census
0,"[129, 194, 887]",8,40,0.115675,24030.691289
1,"[137, 194, 887]",8,31,0.089513,15671.691289


In [100]:
df

,str_ta_list_A,store_count_A,zip_count_A,sales_field_A,kids_0_9_census_A
0,"[129, 194, 887]",8,40,0.115675,24030.691289
1,"[137, 194, 887]",8,31,0.089513,15671.691289
2,"[141, 194, 887]",8,28,0.115208,19956.691289
3,"[116, 194, 887]",8,46,0.102576,20808.691289
4,"[140, 194, 887]",8,30,0.087284,16465.691289
...,...,...,...,...,...
412236,"[129, 135, 135]",8,55,0.108912,30347.000000
412237,"[135, 135, 137]",8,46,0.082749,21988.000000
412238,"[135, 135, 141]",8,43,0.108444,26273.000000
412239,"[116, 135, 135]",8,61,0.095813,27125.000000


In [124]:
df_output=pd.DataFrame()
for ind,row in test.iterrows():
    
    df=test[test['store_count']==row['store_count']].rename(columns={"str_ta_list":"str_ta_list_A",
                                                  "store_count":"store_count_A",
                                                  "zip_count":"zip_count_A",
                                                  "sales_field":"sales_field_A",
                                                  "kids_0_9_census":"kids_0_9_census_A"})
    df['str_ta_list_A']=df['str_ta_list_A'].apply(eval)
    ta_B_list=eval(row['str_ta_list'])
    
    df=df[df['str_ta_list_A'].apply(lambda x: (ta_B_list[1] not in x) & (ta_B_list[0] not in x) & (ta_B_list[2] not in x))]
    
    df['key']=1
    print(df.shape)
    df_B=pd.DataFrame(row).T.rename(columns={"str_ta_list":"str_ta_list_B",
                                                  "store_count":"store_count_B",
                                                  "zip_count":"zip_count_B",
                                                  "sales_field":"sales_field_B",
                                                  "kids_0_9_census":"kids_0_9_census_B"})
    df_B['key']=1
    df=pd.merge(df,df_B,on="key",how="left")

    
    # df['diff_store_count']=abs(df['store_count_A']/df['store_count_B']-1)
    df['diff_zip_count']=abs(df['zip_count_A']/df['zip_count_B']-1)
    df['diff_sales_field']=abs(df['sales_field_A']/df['sales_field_B']-1)
    df['diff_kids_0_9_census']=abs(df['kids_0_9_census_A']/df['kids_0_9_census_B']-1)
    df['score']=df['diff_zip_count']*0.1667+df['diff_sales_field']*0.3333+df['diff_kids_0_9_census']*0.3333
    df=df[df['score']<=0.2]
    
    df_output=df_output.append(df)
    
    if ind%100==1:
        print(ind,datetime.datetime.now(),df_output.shape)


TypeError: eval() arg 1 must be a string, bytes or code object

In [119]:
df_output.sort_values("score")

,str_ta_list_A,store_count_A,zip_count_A,sales_field_A,kids_0_9_census_A,key,str_ta_list_B,store_count_B,zip_count_B,sales_field_B,kids_0_9_census_B,diff_zip_count,diff_sales_field,diff_kids_0_9_census,score
151883,"[182, 206, 301]",8,40,0.115908,24035.000000,1,"[129, 194, 887]",8,40,0.115675,24030.7,0,0.00200983,0.0001793,0.000729638
152181,"[182, 206, 301]",8,40,0.115908,24035.000000,1,"[129, 194, 887]",8,40,0.115675,24030.7,0,0.00200983,0.0001793,0.000729638
26002,"[116, 183, 1149]",8,41,0.135051,29927.518598,1,"[129, 184, 887]",8,41,0.13478,29945,0,0.00200994,0.000583784,0.000864488
142362,"[182, 203, 321]",8,47,0.121570,26634.000000,1,"[116, 184, 887]",8,47,0.121681,26723,0,0.000913783,0.00333046,0.00141461
142627,"[182, 203, 321]",8,47,0.121570,26634.000000,1,"[116, 184, 887]",8,47,0.121681,26723,0,0.000913783,0.00333046,0.00141461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229406,"[175, 202, 221]",8,29,0.133258,29435.000000,1,"[140, 184, 887]",8,31,0.106389,22380,0.0645161,0.252554,0.315237,0.2
201939,"[159, 162, 289]",8,49,0.113448,28452.634014,1,"[141, 184, 887]",8,29,0.134312,25871,0.689655,0.155339,0.0997887,0.2
202435,"[159, 162, 289]",8,49,0.113448,28452.634014,1,"[141, 184, 887]",8,29,0.134312,25871,0.689655,0.155339,0.0997887,0.2
192516,"[175, 202, 333]",8,29,0.141036,36453.000000,1,"[135, 184, 887]",8,46,0.123387,28654,0.369565,0.143043,0.272178,0.2


In [120]:
df_3_combination_M23.shape

(412242, 5)

In [121]:
# 182, 206, 301, 129, 194, 887
# 140, 228, 267, 141, 184, 863
# 177, 215, 347, 116, 193, 1070

,str_ta_list,store_count,zip_count,sales_field,kids_0_9_census
0,"[129, 194, 887]",8,40,0.115675,24030.691289
1,"[137, 194, 887]",8,31,0.089513,15671.691289


In [128]:
# test=df_3_combination_M23.copy()
test['str_ta_list']=test['str_ta_list'].apply(eval)
test=test[test['str_ta_list'].apply(lambda x: (140 not in x) & (228 not in x) & (267 not in x) & (141 not in x) & (184 not in x) & (863 not in x))]
test['str_ta_list']=test['str_ta_list'].astype(str)

In [129]:
df_output=pd.DataFrame()
for ind,row in test.iterrows():
    
    df=test[test['store_count']==row['store_count']].rename(columns={"str_ta_list":"str_ta_list_A",
                                                  "store_count":"store_count_A",
                                                  "zip_count":"zip_count_A",
                                                  "sales_field":"sales_field_A",
                                                  "kids_0_9_census":"kids_0_9_census_A"})
    df['str_ta_list_A']=df['str_ta_list_A'].apply(eval)
    ta_B_list=eval(row['str_ta_list'])
    
    df=df[df['str_ta_list_A'].apply(lambda x: (ta_B_list[1] not in x) & (ta_B_list[0] not in x) & (ta_B_list[2] not in x))]
    
    df['key']=1
    print(df.shape)
    df_B=pd.DataFrame(row).T.rename(columns={"str_ta_list":"str_ta_list_B",
                                                  "store_count":"store_count_B",
                                                  "zip_count":"zip_count_B",
                                                  "sales_field":"sales_field_B",
                                                  "kids_0_9_census":"kids_0_9_census_B"})
    df_B['key']=1
    df=pd.merge(df,df_B,on="key",how="left")

    
    # df['diff_store_count']=abs(df['store_count_A']/df['store_count_B']-1)
    df['diff_zip_count']=abs(df['zip_count_A']/df['zip_count_B']-1)
    df['diff_sales_field']=abs(df['sales_field_A']/df['sales_field_B']-1)
    df['diff_kids_0_9_census']=abs(df['kids_0_9_census_A']/df['kids_0_9_census_B']-1)
    df['score']=df['diff_zip_count']*0.1667+df['diff_sales_field']*0.3333+df['diff_kids_0_9_census']*0.3333
    df=df[df['score']<=0.2]
    
    df_output=df_output.append(df)
    
    if ind%100==1:
        print(ind,datetime.datetime.now(),df_output.shape)


(157136, 6)
(157136, 6)
(157136, 6)


KeyboardInterrupt: 

In [130]:
df_output.sort_values("score")

,str_ta_list_A,store_count_A,zip_count_A,sales_field_A,kids_0_9_census_A,key,str_ta_list_B,store_count_B,zip_count_B,sales_field_B,kids_0_9_census_B,diff_zip_count,diff_sales_field,diff_kids_0_9_census,score
86254,"[177, 215, 347]",8,69,0.131950,32570.294464,1,"[116, 193, 1070]",8,69,0.132363,32565.5,0,0.00311579,0.000148312,0.00108792
85963,"[177, 215, 347]",8,69,0.131950,32570.294464,1,"[116, 193, 1070]",8,69,0.132363,32565.5,0,0.00311579,0.000148312,0.00108792
46148,"[116, 223, 260]",8,54,0.118890,27396.000000,1,"[137, 193, 1070]",8,54,0.119299,27428.5,0,0.00342874,0.00118361,0.0015373
38954,"[116, 223, 260]",8,54,0.118890,27396.000000,1,"[137, 193, 1070]",8,54,0.119299,27428.5,0,0.00342874,0.00118361,0.0015373
7940,"[116, 176, 1309]",8,54,0.120271,27424.644282,1,"[137, 193, 1070]",8,54,0.119299,27428.5,0,0.00814537,0.000139284,0.00276128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74854,"[175, 177, 325]",8,34,0.145009,32895.000000,1,"[137, 193, 1070]",8,54,0.119299,27428.5,0.37037,0.215508,0.199302,0.199997
74795,"[175, 177, 325]",8,34,0.145009,32895.000000,1,"[137, 193, 1070]",8,54,0.119299,27428.5,0.37037,0.215508,0.199302,0.199997
10481,"[135, 172, 529]",8,95,0.150772,41440.000000,1,"[116, 193, 1070]",8,69,0.132363,32565.5,0.376812,0.13908,0.272514,0.199999
156689,"[175, 188, 255]",8,44,0.161023,39154.000000,1,"[116, 193, 1070]",8,69,0.132363,32565.5,0.362319,0.216527,0.202317,0.199999


In [62]:
df_input_M23_3_set_copy_1=df_3_combination_M23.copy()
df_input_M23_3_set_copy_2=df_3_combination_M23.copy()

cols=df_input_M23_3_set_copy_1.columns.tolist()
cols_new=[]
for col in cols:
    cols_new.append(col+"_A")
    cols_new.append(col+"_B")

for col in df_input_M23_3_set_copy_1.columns.tolist():
    df_input_M23_3_set_copy_1=df_input_M23_3_set_copy_1.rename(columns={col:col+"_A"})
    df_input_M23_3_set_copy_2=df_input_M23_3_set_copy_2.rename(columns={col:col+"_B"})
    
df_input_M23_3_set_copy_1['key']=1
df_input_M23_3_set_copy_2['key']=1

df_input_M23_set_of_3=pd.merge(df_input_M23_3_set_copy_1,df_input_M23_3_set_copy_2,on="key",how="outer")
df_input_M23_set_of_3=df_input_M23_set_of_3[cols_new]
gc.collect()

MemoryError: Unable to allocate array with shape (169943466564,) and data type int64

In [ ]:
def intersection_2_list_str(str_l1,str_l2):
    str_l1=set(eval(str_l1))
    str_l2=set(eval(str_l2))
    return str_l1.intersection(str_l2)


In [ ]:
df_input_M23_set_of_3=df_input_M23_set_of_3[df_input_M23_set_of_3['str_ta_list_A']!=df_input_M23_set_of_3['str_ta_list_B']]
gc.collect()


In [ ]:
df_input_M23_set_of_3.shape

In [ ]:
df_input_M23_set_of_3['common_stores']=df_input_M23_set_of_3.apply(lambda x: intersection_2_list_str(x["str_ta_list_A"],x["str_ta_list_B"]), axis=1)
print(datetime.datetime.now())

df_input_M23_set_of_3['common_stores_len']=df_input_M23_set_of_3['common_stores'].apply(len)
print(datetime.datetime.now())


In [ ]:
df_input_M23_set_of_3['common_stores_len'].unique()

In [ ]:
df_input_M23_set_of_3=df_input_M23_set_of_3[df_input_M23_set_of_3['common_stores_len']==0]
df_input_M23_set_of_3.head(4)

In [ ]:
gc.collect()

In [ ]:
cols_metrics=[x for x in cols if x not in ['str_ta_list','Kid_Sales_Tier','common_stores','common_stores_len']]

df_input_M23_set_of_3['score_1']=0
for col_calulate in cols_metrics:
    df_input_M23_set_of_3['diff_'+col_calulate]=df_input_M23_set_of_3[col_calulate+"_A"]/df_input_M23_set_of_3[col_calulate+"_B"]-1
    df_input_M23_set_of_3['diff_'+col_calulate]=df_input_M23_set_of_3['diff_'+col_calulate].apply(abs)
    if "count" in col_calulate:
        df_input_M23_set_of_3['score_1']=df_input_M23_set_of_3['score_1']+df_input_M23_set_of_3['diff_'+col_calulate]*0.1667
    else:
        df_input_M23_set_of_3['score_1']=df_input_M23_set_of_3['score_1']+df_input_M23_set_of_3['diff_'+col_calulate]*0.3333
        
df_input_M23_set_of_3=df_input_M23_set_of_3.sort_values("score_1")

df_input_M23_set_of_3.head(5)

In [ ]:
print(datetime.datetime.now())

In [ ]:
df_input_M23_set_of_3['score_1'].max()

In [ ]:
df_input_M23_set_of_3['score_1'].min()

In [29]:
# df_input_M23_set_of_3.to_csv("./set3_"+str(datetime.datetime.now().date())+".csv",index=False)

In [ ]:
df_input_M23_set_of_3.head(2)

In [ ]:
df_input_M23_set_of_3.shape

In [ ]:
def remove_paired_from_df(df,selected_6_ta_list):
    exp_str=""
    for ta in selected_6_ta_list:
        s1="["+str(ta)+","
        s2=" "+str(ta)+","
        s3=" "+str(ta)+"]"
        exp_str="|".join([exp_str,s1])
        exp_str="|".join([exp_str,s2])
        exp_str="|".join([exp_str,s3])
    exp_str=exp_str[1:]
    print(exp_str)
    df=df[(df["str_ta_list_A"].str.contains(exp_str)==False) & (df["str_ta_list_B"].str.contains(exp_str)==False)]
    return df


In [66]:
set_list_1=[137, 190, 175, 141, 206, 159]
df_1=remove_paired_from_df(df_input_M23_set_of_3,set_list_1)
df_1.shape

[137,| 137,| 137]|[190,| 190,| 190]|[175,| 175,| 175]|[141,| 141,| 141]|[206,| 206,| 206]|[159,| 159,| 159]


(0, 17)

In [57]:
df_1=remove_paired_from_df[~remove_paired_from_df['str_ta_list_A'].str.contains("137")]



(0, 17)

In [58]:
df_1.head(3)

,str_ta_list_A,str_ta_list_B,store_count_A,store_count_B,zip_count_A,zip_count_B,sales_field_A,sales_field_B,kids_0_9_census_A,kids_0_9_census_B,common_stores,common_stores_len,score_1,diff_store_count,diff_zip_count,diff_sales_field,diff_kids_0_9_census


In [53]:
set_list_2=

[141, 206, 159, 137, 190, 175]

In [60]:
df_input_M23_set_of_3.shape

(24619640, 17)

In [69]:
df_input_M23_set_of_3.head(10)

,str_ta_list_A,str_ta_list_B,store_count_A,store_count_B,zip_count_A,zip_count_B,sales_field_A,sales_field_B,kids_0_9_census_A,kids_0_9_census_B,common_stores,common_stores_len,score_1,diff_store_count,diff_zip_count,diff_sales_field,diff_kids_0_9_census
22985065,"[141, 206, 159]","[137, 190, 175]",10,10,35,35,0.130894,0.130917,27917.000000,27950.996976,{},0,0.000464,0.0,0.0,0.000175,0.001216
14129459,"[137, 190, 175]","[141, 206, 159]",10,10,35,35,0.130917,0.130894,27950.996976,27917.000000,{},0,0.000464,0.0,0.0,0.000175,0.001218
27348682,"[116, 179, 162]","[137, 135, 227]",10,10,61,61,0.121161,0.121254,26901.261337,26920.000000,{},0,0.000486,0.0,0.0,0.000761,0.000696
12419078,"[137, 135, 227]","[116, 179, 162]",10,10,61,61,0.121254,0.121161,26920.000000,26901.261337,{},0,0.000486,0.0,0.0,0.000762,0.000697
12320026,"[137, 135, 249]","[116, 140, 348]",10,10,53,53,0.111659,0.111817,31708.000000,31698.000000,{},0,0.000577,0.0,0.0,0.001416,0.000315
23828006,"[116, 140, 348]","[137, 135, 249]",10,10,53,53,0.111817,0.111659,31698.000000,31708.000000,{},0,0.000578,0.0,0.0,0.001418,0.000315
32106800,"[140, 170, 202]","[137, 178, 177]",10,10,39,39,0.104978,0.105104,28806.000000,28823.000000,{},0,0.000594,0.0,0.0,0.001193,0.000590
14284816,"[137, 178, 177]","[140, 170, 202]",10,10,39,39,0.105104,0.104978,28823.000000,28806.000000,{},0,0.000595,0.0,0.0,0.001194,0.000590
26860364,"[116, 178, 188]","[135, 206, 173]",10,10,53,53,0.136547,0.136593,38065.000000,38125.000000,{},0,0.000636,0.0,0.0,0.000335,0.001574
37543708,"[135, 206, 173]","[116, 178, 188]",10,10,53,53,0.136593,0.136547,38125.000000,38065.000000,{},0,0.000637,0.0,0.0,0.000336,0.001576


In [19]:
#2 Sum 4
print("start 4: ",datetime.datetime.now())
list_4_df=[]

for i in range(0,total_ta_len):
    ta_1=M23_TA_list[i]
    k1=dict_store_by_ta_M23[ta_1]
    for j in range(i+1,total_ta_len):
        ta_2=M23_TA_list[j]
        k2=dict_store_by_ta_M23[ta_2]
        
        for k in range(j+1,total_ta_len):
            ta_3=M23_TA_list[k]
            k3=dict_store_by_ta_M23[ta_3]
            
            sum_3=k1+k2+k3
            
            if 6<=sum_3<=12: 
                for l in range(k+1,total_ta_len):
                    ta_4=M23_TA_list[l]
                    k4=dict_store_by_ta_M23[ta_4]
                    sum_num=k1+k2+k3+k4

                    if 10<=sum_num<=13:
                        list_tas_10_13=[ta_1,ta_2,ta_3,ta_4]
                        df=df_input_M23[df_input_M23['TA'].isin(list_tas_10_13)]
                        df['str_ta_list']=str(list_tas_10_13)
                        df=df.groupby("str_ta_list")['store_count','zip_count','sales_field','kids_0_9_census'].sum().reset_index()
                        list_4_df.append(df)
                
            else:
                break
                    
                    
df_4_combination_M23=pd.concat(list_4_df)
del list_4_df
gc.collect()
print("done 4: ",datetime.datetime.now())

start 4:  2019-11-06 14:30:39.552858


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


KeyboardInterrupt: 

In [19]:
54648880440//10**6

54648

In [22]:
sum_3

11

In [24]:
6<=sum_3<=10

False